In [2]:
import pandas as pd
import collections
import numpy as np
import math as mt
import time

In [42]:
df = pd.read_table("/Users/emilybutka/Documents/GitHub/p32_naive_celltag_permutation_test_data.txt")

In [43]:
df.head()

,cluster,celltag
1,0,v1
2,5,NaN
3,0,v2
4,11,NaN
5,0,v1


In [44]:
df['cluster'].value_counts()

0     2678
1     2647
2     1952
3     1930
4     1735
5     1719
6     1592
7     1158
8     1091
9     1024
10     855
11     422
12     421
13     211
Name: cluster, dtype: int64

In [45]:
celltag = df.loc[:,"celltag"]
clusters = df.loc[:,"cluster"]

In [46]:
not_na_ct = pd.notnull(celltag)

In [47]:
celltag_not_na = celltag[not_na_ct]

In [48]:
unique, counts = np.unique(celltag_not_na, return_counts=True)
count_dict = dict(zip(unique, counts))
grt_10_count_dict = {}
#grt_30_count_dict = {}

In [26]:
for key, value in count_dict.items():
    if value > 9:
        grt_10_count_dict[key] = value
    #if value > 30:
    #    grt_30_count_dict[key] = value

In [27]:
len(grt_10_count_dict.keys())

2

In [28]:
grt_10_count_dict

{'v1': 1002, 'v2': 1013}

In [30]:
clusters

1         0
2         5
3         0
4        11
5         0
         ..
19431    11
19432     0
19433     9
19434     7
19435     6
Name: integrated_snn_res.0.8, Length: 19435, dtype: int64

In [31]:
clones = list(grt_10_count_dict.keys())
cluster_list, counts_cls = np.unique(clusters, return_counts=True)
cluster_dict = dict(zip(cluster_list, counts_cls))
cluster_ls = list(cluster_dict.keys())
clones
cluster_ls

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [32]:
def calculate_proportion(sp, cluster_num_list):
    clusters_curr = sp.loc[:,"cluster"]
    unique_curr, counts_curr = np.unique(clusters_curr, return_counts=True)
    count_dict_curr = dict(zip(unique_curr, counts_curr))
    curr_total = sum(count_dict_curr.values())
    cluster_dict = {}
    for c_n in cluster_num_list:
        if c_n in count_dict_curr.keys():
            cluster_dict[c_n] = count_dict_curr[c_n] * 100/curr_total
        else:
            cluster_dict[c_n] = 0
    
    return cluster_dict

In [33]:
#Format: {clone id: {cluster num: percentage, ...}, ...}
clone_null_pct = {}
for c in clones:
    curr_clone_cells = df.loc[df['celltag'] == c]
    clusters_curr = curr_clone_cells.loc[:,"cluster"]
    unique_curr, counts_curr = np.unique(clusters_curr, return_counts=True)
    count_dict_curr = dict(zip(unique_curr, counts_curr))
    curr_total = sum(count_dict_curr.values())
    clone_dict_curr = calculate_proportion(curr_clone_cells, cluster_ls)
    clone_null_pct[c] = clone_dict_curr
    
clone_null_pct

{'v1': {0: 1.6966067864271457,
  1: 20.459081836327346,
  2: 2.4950099800399204,
  3: 10.978043912175648,
  4: 18.562874251497007,
  5: 17.964071856287426,
  6: 4.191616766467066,
  7: 6.586826347305389,
  8: 1.7964071856287425,
  9: 5.289421157684631,
  10: 5.588822355289421,
  11: 0.0998003992015968,
  12: 3.992015968063872,
  13: 0.2994011976047904},
 'v2': {0: 6.910167818361303,
  1: 21.322803553800593,
  2: 2.9615004935834155,
  3: 13.32675222112537,
  4: 8.390918065153011,
  5: 4.244817374136229,
  6: 17.077986179664364,
  7: 1.9743336623889438,
  8: 0.7897334649555775,
  9: 9.970384995064165,
  10: 7.00888450148075,
  11: 0.6910167818361304,
  12: 5.034550839091806,
  13: 0.29615004935834155}}

In [35]:
def randomized_test(clones, orig):
    clone_sample_randoms = {}
    for c_1 in clones:
        curr_clone_cells = orig.loc[df['celltag'] == c_1]
        clone_size = len(curr_clone_cells.index)
        rep_num = mt.ceil(len(orig.index)/clone_size)
        curr_clone_random_percentage = {}
        for rep in range(rep_num):
            # Sample without replacement
            curr_sample = orig.sample(n=clone_size)
            curr_percentage_dict = calculate_proportion(curr_sample, cluster_ls)
            curr_clone_random_percentage[rep] = curr_percentage_dict

        clone_sample_randoms[c_1] = curr_clone_random_percentage
    
    return clone_sample_randoms

In [36]:
replication_number = 1000
time_vec = []
replication_dict = {}
for j in range(replication_number):
    #print(j)
    start_time = time.time()
    curr_replicate_rslt = randomized_test(clones, df)
    end_time = time.time()
    replication_dict[j] = curr_replicate_rslt
    time_vec.append(end_time - start_time)
    #print(end_time - start_time)

0
0.04305100440979004
1
0.040418148040771484
2
0.03616690635681152
3
0.03385758399963379
4
0.03118300437927246
5
0.03152108192443848
6
0.03758406639099121
7
0.03943991661071777
8
0.03356790542602539
9
0.032392263412475586
10
0.0288698673248291
11
0.03312420845031738
12
0.03592085838317871
13
0.03527712821960449
14
0.03669381141662598
15
0.03313159942626953
16
0.0318450927734375
17
0.028966903686523438
18
0.036984920501708984
19
0.035871267318725586
20
0.03290128707885742
21
0.03660392761230469
22
0.03359794616699219
23
0.03705716133117676
24
0.03786015510559082
25
0.03449201583862305
26
0.0338900089263916
27
0.03131699562072754
28
0.03677082061767578
29
0.04420304298400879
30
0.06382012367248535
31
0.04520702362060547
32
0.03575015068054199
33
0.03882884979248047
34
0.06000471115112305
35
0.048825740814208984
36
0.038659095764160156
37
0.032788991928100586
38
0.02869129180908203
39
0.02872610092163086
40
0.03274273872375488
41
0.04438495635986328
42
0.03777503967285156
43
0.03409099578

In [38]:
# Format: {clone id1: {0:[], 1:[], 3:[], 4:[], 6:[]}, clone id2: {0:[], 1:[], 3:[], 4:[], 6:[]}, ...}
rearrange_dict = {}
for k,v in replication_dict.items():
    for sk,sv in v.items():
        curr_c = sk
        curr_pct_dict = {}
        for ssk,ssv in sv.items():
            for sssk,sssv in ssv.items():
                if sssk not in curr_pct_dict.keys():
                    curr_pct_dict[sssk] = [sssv]
                else:
                    curr_pct_dict[sssk].append(sssv)
        #print(len(curr_pct_dict[0]))
        if curr_c not in rearrange_dict.keys():
            rearrange_dict[curr_c] = curr_pct_dict
        else:
            for pct_k,pct_v in curr_pct_dict.items():
                rearrange_dict[curr_c][pct_k].extend(pct_v)

In [39]:
p_val_grt_overall = {}
p_val_less_overall = {}
for key_1,val_1 in rearrange_dict.items():
    curr_cl = key_1
    grt_p = {}
    less_p = {}
    for ky,vl in val_1.items():
        grt_p[ky] = sum(i > clone_null_pct[curr_cl][ky] for i in vl)/len(vl)
        less_p[ky] = sum(j < clone_null_pct[curr_cl][ky] for j in vl)/len(vl)
    p_val_grt_overall[curr_cl] = grt_p
    p_val_less_overall[curr_cl] = less_p

In [40]:
grt_df = pd.DataFrame(p_val_grt_overall)
less_df = pd.DataFrame(p_val_less_overall)

In [41]:
grt_df

,v1,v2
0,1.00000,1.00000
1,0.00000,0.00000
2,1.00000,1.00000
3,0.12135,0.00005
4,0.00000,0.70650
5,0.00000,1.00000
6,1.00000,0.00000
7,0.17585,1.00000
8,1.00000,1.00000
9,0.45145,0.00000
